In [1]:
sf = {
      'account': 'cm18591.west-europe.azure',
      'user': 'LITINGAALTO',
      'password': '199341Asd'
      }
import pandas as pd
from snowflake.sqlalchemy import URL
from sklearn.metrics.pairwise import cosine_similarity
from sqlalchemy import create_engine
import seaborn as sns
import numpy as np

engine = create_engine(URL(
            account = sf['account'],
            user = sf['user'],
            password = sf['password'],
            role = 'SYSADMIN',
            database = 'SEURE_DM_DEV',
            warehouse = 'ANALYTICS_WH',
            schema = 'ANALYTIIKKA',
            numpy = True
        ))
import warnings
warnings.filterwarnings('ignore')

In [2]:
from datetime import date
from dateutil.relativedelta import relativedelta
import requests 
import json 
import math
today=date.today()
toimialaid=[ 10, 20, 30, 40, 60, 70, 90, 100]
# "10 Terveydenhuolto, 20 Varhaiskasvatus, 30 Opetus, 40 Sosiaali, 60 Keittiö ja siivous, 70 Kiinteistö ja valvonta,  90 Hallinto- ja toimisto, 100 Muut"

query=f"select * from VIEW_TOTEUMA_CONFIRM2 where toimiala={toimialaid[0]}"
query1=f"select tyontekijaid, erityisominaisuusid from staging_test.public.STG_KEIKKANETTI_tblTyontekijaErityisominaisuudet\
 where viimeinenvoimassaolopaiva is null or  viimeinenvoimassaolopaiva >='{today}'"  
query2=f"select * from VIEW_TYO_FEATURE2 where toimiala = {toimialaid[0]}"
query4 ='select osoite, postinumero, postitoimipaikka, tyontekija_id from "SEURE_DM_PROD"."LIIKETOIMINTA_HENKILOTASO"."VIEW_D_TYONTEKIJA"  where KEIKKAILUKELPOINEN = True and (osoite is not null and postinumero is not null)'
query5=f"select distinct asiakasid, tyontekijaid from staging_test.public.STG_KEIKKANETTI_tblTyontekijaSulkulista where tyyppi =2 and \
asiakasid in (select distinct id from  staging_test.public.STG_KEIKKANETTI_tblAsiakas where kustannuspaikkaID = {toimialaid[0]})"
query6='select distinct  tyontekijaid,asiakasid from staging_test.public.STG_KEIKKANETTI_tblTyontekijaSulkulista where tyyppi =1'
query7 = "select * from view_varattu_tyontekjia_today"
query8 = "select * from view_tyontekija_ammattinimike"
query9 = f"select * from view_tilaus_lisakriteeri2 where toimiala_id = '{toimialaid[0]}'"

connection = engine.connect()
df = pd.read_sql(query, connection)
df_employee=pd.read_sql(query1, connection)
df_job=pd.read_sql(query2, connection)
emp_add = pd.read_sql(query4, connection)
asiakas_sulkulista = pd.read_sql(query5, connection)
tyontekija_sulkulista = pd.read_sql(query6, connection)
emp_varattu= pd.read_sql(query7, connection)
emp_amma=pd.read_sql(query8, connection)
tilaus_erityisominaisuus= pd.read_sql(query9, connection)
emp_amma.nimi = emp_amma.nimi.apply(lambda x: x.strip())
tilaus_erityisominaisuus = tilaus_erityisominaisuus.fillna(0)
# convert 258 to 351
df_employee.loc[df_employee.erityisominaisuusid==258, 'erityisominaisuusid']=351

### preprocessing

In [3]:
import math
df['nimi']=df['nimi'].apply(lambda x: str(x).strip())
df2019=df[['tyontekijaid','asiakasid','maara','nimi','asiakasnimi','tilausid']]
df2019['job']=df2019['asiakasid'].astype(str)+";"+df2019['nimi']

df2019=df2019.groupby(['tyontekijaid','job'])['maara'].sum().reset_index()
df2019.columns=['tyontekija_id','job','tyovuorot_taytetyt']
df2019.tyontekija_id=df2019.tyontekija_id.astype(int)
df2019 = df2019[df2019['tyovuorot_taytetyt']>0]
df2019['tyovuorot_taytetyt']=df2019['tyovuorot_taytetyt'].apply(lambda x: math.ceil(x/7.5))
df2019['tyovuorot_taytetyt']=df2019['tyovuorot_taytetyt'].astype(int)
n_employees = df2019.tyontekija_id.unique().shape[0]
n_jobs = df2019.job.unique().shape[0]
print("unique employees: {}, unique_jobs: {}".format(n_employees, n_jobs))
df2019.sort_values('tyovuorot_taytetyt', ascending = False).head(3)

unique employees: 5067, unique_jobs: 1631


,tyontekija_id,job,tyovuorot_taytetyt
3206,33598,"7533;Lähihoitaja, vuodeosasto",127
16925,129666,"18973;Lähihoitaja, kotihoito",123
13395,111459,"13776;Lähihoitaja, kotihoito lääkehoitoluvaton",121


In [4]:
# normalize the 5% outliers 
a=df2019.groupby(['tyontekija_id','job']).sum().reset_index().sort_values('tyovuorot_taytetyt',ascending = False)
need_scale=a[a['tyovuorot_taytetyt']>a['tyovuorot_taytetyt'].quantile(0.95)]
rest = a[a['tyovuorot_taytetyt']<=a['tyovuorot_taytetyt'].quantile(0.95)]
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler(feature_range=(a['tyovuorot_taytetyt'].quantile(0.95),a['tyovuorot_taytetyt'].quantile(0.95)+8))
scaler.fit(need_scale['tyovuorot_taytetyt'].values.reshape(-1,1))
need_scale['tyovuorot_taytetyt']=scaler.transform(need_scale['tyovuorot_taytetyt'].values.reshape(-1,1))
# append the data with the modified 5%
df_scaled=rest.append(need_scale)
df_scaled['jobs']=df_scaled['job'].apply(lambda x: str(x).strip())
df_scaled['job']=df_scaled['jobs']
ratings=df_scaled.pivot_table(index='tyontekija_id', columns='job', values='tyovuorot_taytetyt').fillna(0)

In [5]:
# This is the jobs that have orders in past 6kk, terveydenhuolto
df_terv=df_job
df_terv['job']=df_terv['job'].apply(lambda x: str(x).strip())
df['job']=df['asiakasid'].astype(str)+";"+df['nimi']

terv2019=pd.DataFrame(df.groupby(['tyontekijaid','job'])['maara'].sum()).reset_index()
df_terv=terv2019.merge(df_terv[['job', 'katuosoite','postinumero','postitoimipaikka','erityisominaisuusid']], how='left', on='job')
a=df_terv[['job', 'erityisominaisuusid']]
a.fillna(0, inplace=True)
a['arvo']=1
tyo_taidot=a.pivot_table(index='job', columns='erityisominaisuusid').fillna(0)
tyo_taidot=tyo_taidot['arvo'].drop(0,1)
print('there are {} jobs and {} erityisominaisuusid in tyo_taidot'.format(tyo_taidot.shape[0], tyo_taidot.shape[1]))

there are 1632 jobs and 61 erityisominaisuusid in tyo_taidot


In [6]:
b=df_employee[['tyontekijaid','erityisominaisuusid']]
b['arvo']=1
x=b.pivot_table(index='tyontekijaid', columns='erityisominaisuusid').fillna(0)['arvo']
c=list(set(tyo_taidot.columns) & set(x.columns))
if len(list(set(tyo_taidot.columns)-set(c))) != 0:
    print('None of the employees have {} skills as required in jobs'.format(list(set(tyo_taidot.columns)-set(c))))
tyo_taidot=tyo_taidot[c]
employee_taidot=x[c]
#These are the employees that are missing erityisominaisuus
bb=set(df.tyontekijaid.unique())- set(employee_taidot.index.tolist())
print('Employee(s) that do not have any features: {}'.format(bb))
# Add the missing employess to this employee matrix
employee_taidot=employee_taidot.append(pd.DataFrame(np.zeros((len(bb),employee_taidot.shape[1])), index=list(set(df.tyontekijaid.unique())- set(employee_taidot.index.tolist())), columns=c))
print('there are {} employees and {} erityisominaisuusid in employee feature table'.format(x.shape[0], x.shape[1]))
print('filtering only common erityisominaisuusid with job+add missing employees...\nthere are {} employees and {} common erityisominaisuusid in employee feature table'.format(employee_taidot.shape[0], employee_taidot.shape[1]))

Employee(s) that do not have any features: set()
there are 54332 employees and 438 erityisominaisuusid in employee feature table
filtering only common erityisominaisuusid with job+add missing employees...
there are 54332 employees and 61 common erityisominaisuusid in employee feature table


### Feature data handling

### Load coordinates for employees and customers & sulkulista

In [7]:
xs=pd.read_excel('employee_cod.xlsx')
xs=xs.drop('Unnamed: 0',1)
xs.columns=['tyontekijaid','address','lat','lon']
coordinate_dictionary={}
for _,i in xs.iterrows():
    coordinate_dictionary[i.tyontekijaid]= [i.address, i.lat, i.lon]
employee_cod=pd.DataFrame.from_dict(coordinate_dictionary, orient='index')
employee_cod.columns=['address','lat','lon']
a=pd.read_excel('customer_cod.xlsx')
customer_cod=a[['ASIAKAS_ID','katunimi_tulos','x','y' ]].set_index('ASIAKAS_ID')
customer_cod.columns=['address','lat','lon']
# Make customer dictionary
coordinate_dictionary_customer={}
for _,i in pd.read_excel('customer_cod.xlsx').iterrows():
    coordinate_dictionary_customer[i.ASIAKAS_ID]= [i.x, i.y]

In [8]:
asiakas_sulkulista['arvo']=1
asiakas_sulkulista=asiakas_sulkulista.pivot_table(index='asiakasid', columns='tyontekijaid').fillna(0)['arvo']
tyontekija_sulkulista['arvo']=1
tyontekija_sulkulista=tyontekija_sulkulista.pivot_table(index='tyontekijaid', columns='asiakasid').fillna(0)['arvo']

### Three models

In [9]:
from sklearn.neighbors import NearestNeighbors
from scipy.sparse.linalg import svds
from math import sin, cos, sqrt, atan2, radians
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity
import statistics
import math
# Median distance is round to upper int


class employee_recommendation:
    def __init__(self,  ratings, employee_taidot,tyo_taidot,coordinate_dictionary,
                 coordinate_dictionary_customer,tyontekija_sulkulista,  emp_amma):
        self.__ratings=ratings
        self.__employee_taidot=employee_taidot
        self.__tyo_taidot=tyo_taidot
        self.__coordinate_dictionary=coordinate_dictionary
        self.__coordinate_dictionary_customer=coordinate_dictionary_customer
        self.__tyontekija_sulkulista=tyontekija_sulkulista
        self.__emp_amma=emp_amma 
        
    def sim_recommendation(self, userid):
        """
        :userid: employeeid
        :return: recommendation in dataframe format
        * if employee did not do gig in past 6kk, then return empty list
        """
        similarities=[]
        indices=[]
        recommendation=[]
        model_knn = NearestNeighbors(metric = 'cosine', algorithm = 'brute') 
        model_knn.fit(self.__ratings)
        try:
            distances, indices = model_knn.kneighbors(self.__ratings.loc[userid].values.reshape(1, -1), n_neighbors = 30)
            similarities = 1-distances.flatten()
            similarities1=similarities[similarities>=0.5]
            if len(similarities1)<2:
                recommendation=pd.DataFrame(self.__ratings.loc[userid][self.__ratings.loc[userid]>0]).reset_index()
                recommendation.columns=['job','shifts']
                return recommendation
            else:
                x=0
                for i in range(len(similarities1)):
                    user_i_demean = self.__ratings.iloc[indices[0][i]]
                    user_i_cal = user_i_demean * similarities1[i] 
                    x += user_i_cal
                list1=list(zip([j for j in x.values], [j for j in x.index]))
                for j in sorted(list1, key = lambda x: x[0], reverse = True)[:30]:
                    if j[0]>0:
                        recommendation.append((j[1],j[0]))
            recommendation=pd.DataFrame(recommendation, columns=['job', 'shifts'])
            return recommendation
        except:
            return []
        
    
    def svd(self):
        """
        :return employee-job matrix with SVD aggregation
        """
        R = self.__ratings.values
        user_ratings_mean = np.mean(R, axis = 1)
        R_demeaned = R - user_ratings_mean.reshape(-1, 1)
        U, sigma, Vt = svds(R_demeaned, k = 50)
        sigma = np.diag(sigma)

        all_user_predicted_ratings = np.dot(np.dot(U, sigma), Vt) + user_ratings_mean.reshape(-1, 1)
        preds_df = pd.DataFrame(all_user_predicted_ratings, columns = self.__ratings.columns, index=self.__ratings.index)
        a=pd.DataFrame(preds_df.values.astype('int'),columns=preds_df.columns, index=preds_df.index)
        return a
    
    def svd_recommendation(self, userid):
        """
        :userid: employeeid
        :return: recommendation in dataframe format
        * if employee did not do gig in past 6kk, then return empty list
        """

        preds_df=self.svd()
        try:
            a=pd.DataFrame(preds_df.loc[userid].sort_values(ascending=False)[:30]).reset_index()
            x=a[a[userid]>0.5]
            x.columns=['job', 'shifts']
            if x.shape[0]==0:
                return pd.DataFrame(columns=['job','shifts'])
            return x
        except:
            print('This user did not do any jobs in the past 6kk')
            return []
            
    def cf(self,userid, k=10):
        """
        :userid: employeeid
        :k: number of recommendation shown
        return: return combination of KNN and SVD recommendation in dataframe format
        * if employee did not do gig in past 6kk, then return empty list
        """
        sim=self.sim_recommendation(userid)
        svd=self.svd_recommendation(userid)
        try:
            svd_job=svd.job.unique().tolist()
            sim_job=sim.job.unique().tolist()
            common_jobs=(set(svd_job) & set(sim_job))
            print('based on historical shift fulfillment, recommendation:\n')

            svd_others=[i for i in list(set(svd_job) - set(common_jobs))]
            sim_others=[i for i in list(set(sim_job) - set(svd_job)) ]

            b=svd[svd.job.isin(svd_others)].append(sim[sim.job.isin(sim_others)]).sort_values('shifts', ascending=False)
            cf_combined=svd[svd.job.isin(common_jobs)].append(b)
            if cf_combined.shape[0]<k:
                print(cf_combined.job.reset_index().drop('index',1))
                return cf_combined.job.reset_index().drop('index',1)
            else:
                print(cf_combined[:k].job.reset_index().drop('index',1))
                return cf_combined[:k].job.reset_index().drop('index',1)
        except:
            return []
        

    def validate_skill(self, userid):
        """
        :userid: employeeid
        :return: validated recommendation from previous step (self.cf) in dataframe format
        * if employee did not do gig in past 6kk, then return empty list
        """
        try:
            eligible=self.__emp_amma[self.__emp_amma['tyontekijaid']==userid].nimi.unique().tolist()
            
            xx=self.__employee_taidot.loc[userid].values.reshape(-1,1)
            a=self.__tyo_taidot.dot(xx).reset_index()
            a['threshold']=self.__tyo_taidot.sum(1).tolist()
            a['amma']=a.job.apply(lambda x: x.split(';')[1])
            a=a[a['amma'].isin(eligible)]
            
            return a[a[0]>=a['threshold']].sort_values(0, ascending=False)
        except:
            print('this employee has no skill and did not do gig in the past 6kk in this industry')
            return []

   
    def calculate_station_distance(self, userid, lon2, lat2 ):
        """
        :userid: employeeid
        :lon2: longnitude (of each customer)
        :lat2: lagnitude (of each customer)
        :return: calculated distance between the employee and customers
        * if employee address or customer address missing, then return empty list
        """
        try:

            lon1 = radians(self.__coordinate_dictionary[userid][2])
            lat1 = radians(self.__coordinate_dictionary[userid][1])

            a = math.sin(( lat1- lat2) / 2)**2 + math.cos(lat1) * math.cos(lat2) * math.sin((lon1-lon2) / 2)**2
            c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
            distance = 6373 * c
            return distance
        except:
            print(" Can't find this employee's address!")
            return []


    def employee_median_distance(self, userid):
        """
        :userid: employeeid
        :return: calculated median distance of the employee in past 6 months
        """
        km=[]
        a=self.__ratings.loc[userid][self.__ratings.loc[userid]>0]
        checklist=pd.DataFrame(a).reset_index().job.apply(lambda x: int(x.split(';')[0]))
        for i in checklist:
            if i in coordinate_dictionary_customer.keys():
                    lat2, lon2= radians(self.__coordinate_dictionary_customer[i][0]), radians(self.__coordinate_dictionary_customer[i][1])
                    distance = self.calculate_station_distance(userid, lon2, lat2)
                    km.append(distance)
        return math.ceil(statistics.median(km))

    def content_based(self,userid, km=None):
        """
        :userid: employeeid
        :km: maximum distance between employee and job, default is None
        :return: content-based recommendation in dataframe format within the defined distance (km)
        """
        ok=[]
        if km == None:
            try:
                km=self.employee_median_distance(userid)
                print('The median distance of this employee in last 6kk is: {}'.format(km))
            except:
                km=10
        try:
            b=self.validate_skill(userid)
            for _, i in b.iterrows():
                if int(i.job.split(';')[0]) in self.__coordinate_dictionary_customer.keys():
                    lat2, lon2= radians(self.__coordinate_dictionary_customer[int(i.job.split(';')[0])][0]), radians(self.__coordinate_dictionary_customer[int(i.job.split(';')[0])][1])
                    distance = self.calculate_station_distance(userid, lon2, lat2 )

                    if distance <=km:
                        ok.append(i.job)
            return b[b.job.isin(ok)]
        except:
            try:
                b=self.validate_skill(userid)
                return b
            except:
                b=self.__tyo_taidot[self.__tyo_taidot.sum(1)==0].index.tolist()
                eligible=self.__emp_amma[self.__emp_amma['tyontekijaid']==userid].nimi.unique().tolist()
                b=[x for x in b if x.split(';')[1] in eligible]
                if userid in self.__coordinate_dictionary.keys():
                    for i in b:
                        if int(i.split(';')[0]) in self.__coordinate_dictionary_customer.keys():
                            lat2, lon2= radians(self.__coordinate_dictionary_customer[int(i.split(';')[0])][0]), radians(self.__coordinate_dictionary_customer[int(i.split(';')[0])][1])
                            distance = self.calculate_station_distance(userid, lon2, lat2 )
                            if distance <=km:
                                ok.append(i)
                    c=pd.DataFrame(ok, columns=['job'])
                    c[0]=0
                    c['threshold']=0
                    return c
                else:
                    c=pd.DataFrame(b, columns=['job'])
                    c[0]=0
                    c['threshold']=0
                    return c

    def hybrid_recommendation_w_sulku(self,userid, k=10, km=None):
        """
        :userid: employeeid
        :k: number of recommendation shown (default: 10)
        :km: maximum distance between employee and job, default is None
        :return: hybrid recommendation (combining all models) in dataframe format within the defined distance (km)
        """
        sim=self.sim_recommendation(userid)
        svd=self.svd_recommendation(userid)
        content=self.content_based(userid,km)

        try:
            svd_job=svd.job.unique().tolist()
            sim_job=sim.job.unique().tolist()
            common_jobs=(set(svd_job) & set(sim_job))

            eligible_common_jobs = [i for i in common_jobs if i in self.validate_skill(userid).job.tolist()]
            print('{} recommended jobs for employee {}\n'.format(k,userid))

            svd_others=[i for i in list(set(svd_job) - set(common_jobs)) if i in self.validate_skill(userid).job.tolist()]
            sim_others=[i for i in list(set(sim_job) - set(svd_job)) if i in self.validate_skill(userid).job.tolist()]

            b=svd[svd.job.isin(svd_others)].append(sim[sim.job.isin(sim_others)]).sort_values('shifts', ascending=False)
            if userid not in self.__tyontekija_sulkulista.index.tolist():
                cf_combined=svd[svd.job.isin(eligible_common_jobs)].append(b)
                if cf_combined.shape[0]<k:
                    combined=pd.DataFrame(cf_combined).append(pd.DataFrame(content[:int(k-cf_combined.shape[0])].job)).reset_index().drop('index',1).fillna(0)
                    xx=self.__employee_taidot.loc[userid,].values.reshape(-1,1)
                    yy=self.__tyo_taidot.loc[combined.job.tolist()]
                    a=yy.dot(xx).reset_index()     
                    combined['threshold']=a[0]
                    b=combined.sort_values(['threshold','shifts'], ascending=False)
                    if b.shape[0]>=k:
                        b=b.drop_duplicates('job')
                        print(b[:k].job.reset_index().drop('index',1))
                        return b[:k].job.reset_index().drop('index',1)
                    else:
                        b=b.drop_duplicates('job')
                        print(b.job.reset_index().drop('index',1))
                        return b.job.reset_index().drop('index',1)
                else:
                    cfc=cf_combined[:int(cf_combined.shape[0]*0.7)]
                    combined=cfc.append(pd.DataFrame(content[:min(3,int(cfc.shape[0]/4))].job)).reset_index().drop('index',1).fillna(0)
                    xx=self.__employee_taidot.loc[userid,].values.reshape(-1,1)
                    yy=self.__tyo_taidot.loc[combined.job.tolist()]
                    a=yy.dot(xx).reset_index()   
                    combined['threshold']=a[0]
                    b=combined.sort_values(['threshold','shifts'], ascending=False)
                    if b.shape[0]>=k:
                        b=b.drop_duplicates('job')
                        print(b[:k].job.reset_index().drop('index',1))
                        return b[:k].job.reset_index().drop('index',1)
                    else:
                        b=b.drop_duplicates('job')
                        print(b.job.reset_index().drop('index',1))
                        return b.job.reset_index().drop('index',1)

            else:
                searchfor =[str(i) for i in self.__tyontekija_sulkulista.loc[userid][self.__tyontekija_sulkulista.loc[userid]>0].index.tolist()]
                cf_combined=svd[svd.job.isin(eligible_common_jobs)].append(b)
                if cf_combined.shape[0]<k:
                    combined=pd.DataFrame(cf_combined).append(pd.DataFrame(content[:int(k-cf_combined.shape[0])].job)).reset_index().drop('index',1).fillna(0)
                    xx=self.__employee_taidot.loc[userid,].values.reshape(-1,1)
                    yy=self.__tyo_taidot.loc[combined.job.tolist()]
                    a=yy.dot(xx).reset_index()   
#                     combined['threshold']=a[0]
                    combined=combined.merge(a, on='job')[['job','threshold','shifts']]
                    b=combined.sort_values(['threshold','shifts'], ascending=False)  
                    b=b[~b.job.str.contains('|'.join(searchfor))]
                    b=b.sort_values(['threshold','shifts'], ascending=False)
                    if b.shape[0]>=k:
                        b=b.drop_duplicates('job')
                        print(b[:k].job.reset_index().drop('index',1).drop_duplicates())
                        return b[:k].job.reset_index().drop('index',1).drop_duplicates()
                    else:
                        b=b.drop_duplicates('job')
                        print(b.job.reset_index().drop('index',1).drop_duplicates())
                        return b.job.reset_index().drop('index',1).drop_duplicates()
                else:
                    cfc=cf_combined[:int(cf_combined.shape[0]*0.7)]
                    combined=cfc.append(pd.DataFrame(content[:min(3,int(cfc.shape[0]/4))].job)).reset_index().drop('index',1).fillna(0)
                    xx=self.__employee_taidot.loc[userid,].values.reshape(-1,1)
                    yy=self.__tyo_taidot.loc[combined.job.tolist()]
                    a=yy.dot(xx).reset_index()  
#                     combined['threshold']=a[0]
                    combined=combined.merge(a, on='job')[['job','threshold','shifts']]
                    b=combined.sort_values(['threshold','shifts'], ascending=False)
                    b=b[~b.job.str.contains('|'.join(searchfor))]
                    b=b.sort_values(['threshold','shifts'], ascending=False)
                    if b.shape[0]>=k:
                        b=b.drop_duplicates('job')
                        print(b[:k].job.reset_index().drop('index',1).drop_duplicates())
                        return b[:k].job.reset_index().drop('index',1).drop_duplicates()
                    else:
                        b=b.drop_duplicates('job')
                        print(b.job.reset_index().drop('index',1).drop_duplicates())
                        return b.job.reset_index().drop('index',1).drop_duplicates()

        except:
            try:
                a=self.content_based(userid,km)
                if a.shape[0]>=k:
                    print(a[:k].job.reset_index().drop('index',1).drop_duplicates())
                    return a[:k].job.reset_index().drop('index',1).drop_duplicates()
                else:
                    print(b.job.reset_index().drop('index',1).drop_duplicates())
                    return b.job.reset_index().drop('index',1).drop_duplicates()
            except:
                print('failed')
                return []

In [10]:
a=employee_recommendation(ratings, employee_taidot,tyo_taidot,coordinate_dictionary,
                 coordinate_dictionary_customer,tyontekija_sulkulista,  emp_amma)

In [11]:
ass=a.hybrid_recommendation_w_sulku(106)
ass

The median distance of this employee in last 6kk is: 23
10 recommended jobs for employee 106

                                       job
0             5329;Lähihoitaja, vammaistyö
1   4850;Lähihoitaja, psykiatria ja päihde
2  19355;Lähihoitaja, perusterveydenhuolto
3             7845;Lähihoitaja, vammaistyö
4             9624;Lähihoitaja, vammaistyö
5             5254;Lähihoitaja, vammaistyö
6             9204;Lähihoitaja, vammaistyö
7            19196;Lähihoitaja, vammaistyö
8            10358;Lähihoitaja, vammaistyö
9             8980;Lähihoitaja, vammaistyö


,job
0,"5329;Lähihoitaja, vammaistyö"
1,"4850;Lähihoitaja, psykiatria ja päihde"
2,"19355;Lähihoitaja, perusterveydenhuolto"
3,"7845;Lähihoitaja, vammaistyö"
4,"9624;Lähihoitaja, vammaistyö"
5,"5254;Lähihoitaja, vammaistyö"
6,"9204;Lähihoitaja, vammaistyö"
7,"19196;Lähihoitaja, vammaistyö"
8,"10358;Lähihoitaja, vammaistyö"
9,"8980;Lähihoitaja, vammaistyö"
